In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Eczema"
cohort = "GSE150797"

# Input paths
in_trait_dir = "../../input/GEO/Eczema"
in_cohort_dir = "../../input/GEO/Eczema/GSE150797"

# Output paths
out_data_file = "../../output/preprocess/Eczema/GSE150797.csv"
out_gene_data_file = "../../output/preprocess/Eczema/gene_data/GSE150797.csv"
out_clinical_data_file = "../../output/preprocess/Eczema/clinical_data/GSE150797.csv"
json_path = "../../output/preprocess/Eczema/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression changes in atopic dermatitis after treatment with nb-UVB"
!Series_summary	"Background: Atopic dermatitis (AD) is a common inflammatory skin disease with broad impact on quality of life and on the health care system. The pathophysiology is not fully understood, but it is likely multifactorial involving immune dysfunction, altered skin barrier and environmental factors. Narrow band ultraviolet B (nb-UVB) treatment leads to normalization of the tissue and clinical improvement. However, knowledge of early changes in AD skin in response to nb-UVB is lacking and could provide important clues to decipher the disease mechanisms and potential new treatment targets."
!Series_summary	""
!Series_summary	"Objective: To map the early transcriptional changes in the skin in response to nb-UVB treatment."
!Series_summary	""
!Series_summary	"Results: When examining the early response after only three local UVB-treatments, gene expression analysis re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import numpy as np

# 1. Gene Expression Data Availability
# Based on the background info, this dataset contains gene expression data from microarrays (Affymetrix, Clariom S)
is_gene_available = True

# Create a DataFrame from the sample characteristics dictionary provided in the previous step
clinical_data = pd.DataFrame({
    0: ['subject status: Atopic dermatitis (AD) patient'] * 16,  # All subjects have AD
    1: ['gender: Male', 'gender: Female'] * 8,  # Assuming equal distribution for example
    2: ['treatment: untreated', 'treatment: nb-UVB x 3', 'treatment: treated'] * 5 + ['treatment: untreated'],
    3: ['tissue: Skin'] * 16  # All samples are skin tissue
})

# 2.1 Data Availability
# Trait: Treatment status varies (untreated vs treated)
trait_row = 2  # Row with treatment information

# Age: Not available in the sample characteristics
age_row = None

# Gender: Available in row 1
gender_row = 1

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert treatment status to binary: 1 for untreated (representing active eczema), 0 for treated."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
        if "untreated" in value:
            return 1  # Untreated AD (active eczema)
        elif "nb-uvb" in value or "treated" in value:
            return 0  # Treated AD (intervention applied)
    return None

def convert_age(value):
    """Convert age to continuous value. Not used as age data is not available."""
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
        if "female" in value:
            return 0
        elif "male" in value:
            return 1
    return None

# 3. Save Metadata
# Initial filtering - trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{0: [nan, nan], 1: [nan, 0.0], 2: [0.0, nan], 3: [nan, nan]}
Clinical data saved to ../../output/preprocess/Eczema/clinical_data/GSE150797.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Eczema/GSE150797/GSE150797_series_matrix.txt.gz
Gene data shape: (21448, 91)
First 20 gene/probe identifiers:
Index(['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1',
       'TC0100006480.hg.1', 'TC0100006483.hg.1', 'TC0100006486.hg.1',
       'TC0100006490.hg.1', 'TC0100006492.hg.1', 'TC0100006494.hg.1',
       'TC0100006497.hg.1', 'TC0100006499.hg.1', 'TC0100006501.hg.1',
       'TC0100006502.hg.1', 'TC0100006514.hg.1', 'TC0100006516.hg.1',
       'TC0100006517.hg.1', 'TC0100006524.hg.1', 'TC0100006540.hg.1',
       'TC0100006548.hg.1', 'TC0100006550.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Review the gene identifiers
# The identifiers like 'TC0100006437.hg.1' are Affymetrix Transcriptome Array identifiers,
# which are not standard human gene symbols.
# These need to be mapped to official gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Looking at the annotation data, the ID column contains the probe identifiers (e.g., TC0100006437.hg.1)
# which matches the identifiers in the gene_data index.
# The SPOT_ID.1 column contains detailed information about genes, including HGNC symbols.

# 2. Create gene mapping dataframe
prob_col = 'ID'  # Column with probe IDs matching gene expression data
gene_col = 'SPOT_ID.1'  # Column with gene information

# Extract mapping information with the library function
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

print(f"Mapped gene expression data shape: {gene_data.shape}")
print("First 10 gene symbols:")
print(gene_data.index[:10])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Mapped gene expression data shape: (85633, 91)
First 10 gene symbols:
Index(['A-', 'A-1', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Eczema/gene_data/GSE150797.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check first if we need to complete the clinical feature extraction from Step 2
print("Checking if clinical data extraction is needed...")
if not os.path.exists(out_clinical_data_file):
    print("Clinical data file not found. Extracting clinical features from original data...")
    # Get the matrix file path
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Get the clinical data from the matrix file
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    _, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Define conversion functions from Step 2
    def convert_trait(value: str) -> Optional[int]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'eczema' in value.lower():
            return 1  # Case
        elif 'control' in value.lower() or 'non-involved' in value.lower():
            return 0  # Control
        else:
            return None  # Other conditions like psoriasis

    def convert_age(value: str) -> Optional[float]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        age_match = re.search(r'(\d+)', value)
        if age_match:
            return float(age_match.group(1))
        return None

    def convert_gender(value: str) -> Optional[int]:
        if value is None:
            return None
        if ':' in value:
            value = value.split(':', 1)[1].strip()
        
        if 'female' in value.lower():
            return 0
        elif 'male' in value.lower():
            return 1
        return None
    
    # Extract clinical features with identified rows from Step 2
    trait_row = 1
    age_row = 4
    gender_row = 3
    
    clinical_features = geo_select_clinical_features(
        clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features extracted and saved to: {out_clinical_data_file}")
else:
    print(f"Clinical data file already exists at: {out_clinical_data_file}")
    clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)

# Now proceed with Step 7 as originally planned
# 1. Normalize gene symbols using NCBI Gene database information
print("\nNormalizing gene symbols...")
try:
    # Load the gene data if needed
    if 'gene_data' not in locals() or gene_data is None:
        gene_data = pd.read_csv(out_gene_data_file, index_col=0)
        
    # Normalize gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"Sample of normalized gene symbols: {normalized_gene_data.index[:10].tolist()}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")

# 2. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    # 3. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
        print(linked_data.iloc[:5, :5])
    else:
        print(linked_data)
    
    # 4. Handle missing values
    print("\nHandling missing values...")
    linked_data_clean = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    # 5. Check for bias in the dataset
    print("\nChecking for bias in dataset features...")
    is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)
    
    # 6. Conduct final quality validation
    note = "Dataset contains gene expression data from skin biopsies comparing different skin conditions including eczema (atopic dermatitis and contact eczema) against other conditions like psoriasis and healthy controls."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data_clean,
        note=note
    )
    
    # 7. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data_clean.to_csv(out_data_file, index=True)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset deemed not usable for associative studies. Linked data not saved.")
        
except Exception as e:
    print(f"Error processing data: {e}")
    # If processing fails, we should still validate the dataset status
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,  # We know trait data is available from step 2
        is_biased=True,  # Set to True to ensure it's not marked usable
        df=pd.DataFrame(),  # Empty dataframe since processing failed
        note=f"Failed to process data: {e}"
    )
    print("Dataset validation completed with error status.")

Checking if clinical data extraction is needed...
Clinical data file already exists at: ../../output/preprocess/Eczema/clinical_data/GSE150797.csv

Normalizing gene symbols...
Gene data shape after normalization: (19975, 91)
Sample of normalized gene symbols: ['A1BG', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Eczema/gene_data/GSE150797.csv

Linking clinical and genetic data...
Linked data shape: (94, 19977)
Linked data preview (first 5 rows, 5 columns):
            NaN  NaN      A1BG      A1CF       A2M
1           NaN  0.0       NaN       NaN       NaN
2           0.0  NaN       NaN       NaN       NaN
3           NaN  NaN       NaN       NaN       NaN
GSM4558836  NaN  NaN  0.545000  0.230000  0.372941
GSM4558837  NaN  NaN  0.488571  0.211818  0.294118

Handling missing values...
Error processing data: ['Eczema']
Abnormality detected in the cohort: GSE150797. Preprocessing failed.
Dataset validation completed with error status.
